In [34]:
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import calendar
import import_ipynb
import nettoyage_bases

In [52]:
def conso_moy_gaz():
    consogaz = nettoyage_bases.conso_gaz()
    conso_moyenne_gaz=consogaz[['Date','Consogaz_ag (MW)']].groupby('Date').mean()
    conso_moyenne_gaz=conso_moyenne_gaz.reset_index(level='Date')
    conso_moyenne_gaz=conso_moyenne_gaz.sort_values(by='Date')
    return conso_moyenne_gaz

def conso_gaz_ete():
    consogaz = nettoyage_bases.conso_gaz()
    consogaz_ete=consogaz[(consogaz['Mois']<9) & (consogaz['Mois']>5)]
    return consogaz_ete

def conso_gaz_hiver():
    consogaz = nettoyage_bases.conso_gaz()
    consogaz_hiver=consogaz[(consogaz['Mois']<4) | (consogaz['Mois']==12) | (consogaz['Mois']==11)]
    return consogaz_hiver

In [74]:
def conso_moy_elec():
    consoelec = nettoyage_bases.conso_elec()
    conso_moyenne_elec=consoelec[['Date','Consoelec_ag (MW)']].groupby('Date').mean()
    conso_moyenne_elec=conso_moyenne_elec.reset_index(level='Date')
    conso_moyenne_elec=conso_moyenne_elec.sort_values(by='Date')
    return conso_moyenne_elec

def conso_elec_ete():
    consoelec = nettoyage_bases.conso_elec()
    consoelec_ete=consoelec[(consoelec['Mois']<9) & (consoelec['Mois']>5)]
    return consoelec_ete

def conso_elec_hiver():
    consoelec = nettoyage_bases.conso_elec()
    consoelec_hiver=consoelec[(consoelec['Mois']<4) | (consoelec['Mois']==12) | (consoelec['Mois']==11)]
    return consoelec_hiver

In [75]:
def plot_conso_Fr_2020():

    def lissage(Lx,Ly,p):
        '''Fonction qui débruite une courbe par une moyenne glissante
        sur 2P+1 points'''
        Lxout = []
        Lyout = []
        Lxout = Lx[p: -p]
        for index in range(p, len(Ly)-p):
            average = np.mean(Ly[index-p : index+p+1])
            Lyout.append(average)
        return [Lxout,Lyout]


    

#On lisse car les données ne sont pas identiquement distribués
    Tab_Date=['2020-01-09','2020-03-01','2020-05-01','2020-07-01','2020-09-01','2020-11-01','2020-12-31']
    Tab_Date2=['01-Janv','01-Mars','01-Mai','01-Jui','01-Sept','01-Nov','31-Dec']



    conso_moyenne_gaz = conso_moy_gaz()
    lissgaz = lissage(conso_moyenne_gaz['Date'],conso_moyenne_gaz['Consogaz_ag (MW)'],3)

    conso_moyenne_elec=conso_moy_elec()
    lisselec = lissage(conso_moyenne_elec['Date'],conso_moyenne_elec['Consoelec_ag (MW)'],3)

    plt.close()
    plt.plot(lissgaz[0],lissgaz[1],label='Gaz')
    plt.plot(lisselec[0],lisselec[1],label='Electricité')
    plt.title('Evolution de la consommation en gaz et electricité en France en 2020')
    plt.xticks(Tab_Date,Tab_Date2)
    plt.xlabel('Date')
    plt.ylabel('Consommation (MW)')
    plt.legend()
    return plt.show()

In [77]:
def cor_elec_gaz():
    conso_moyenne_gaz = conso_moy_gaz()
    conso_moyenne_elec = conso_moy_elec()
#######  Il y a visiblement une corrélation entre la consommation de gaz et d'éléctricité, on verifie ca : 
    corr_elec_gaz=pearsonr(conso_moyenne_gaz['Consogaz_ag (MW)'],conso_moyenne_elec['Consoelec_ag (MW)'])
    return corr_elec_gaz

In [92]:
def stat_des_conso_m():
    conso_moyenne_gaz = conso_moy_gaz()
    conso_moyenne_elec = conso_moy_elec()
    df = pd.concat([conso_moyenne_gaz.describe() , conso_moyenne_elec.describe()] , axis=1)
    df.columns = ['Consommation agrégée en gaz en MW' , 'Consommation agrégée en électricité en MW']
    return df

In [97]:
def plot_conso_gaz_ete():
    
    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}

    def pond_pop(x):
        return Pop_region[x]    

    consogaz_ete= conso_gaz_ete()
    consoelec_ete= conso_elec_ete()

    consogaz_ete['Consogaz/hab']=consogaz_ete['Consommation brute gaz (W)']/consogaz_ete['Region'].transform(pond_pop)
    conso_moy_gaz_heure_pond_ete=consogaz_ete[['Heure','Consogaz/hab']].groupby('Heure').mean()
    conso_moy_gaz_heure_pond_ete=conso_moy_gaz_heure_pond_ete.reset_index(level='Heure')

    consoelec_ete['Consoelec/hab']=consoelec_ete['Consommation brute elec (W)']/consoelec_ete['Region'].transform(pond_pop)
    conso_moy_elec_heure_pond_ete=consoelec_ete[['Heure','Consoelec/hab']].groupby('Heure').mean()
    conso_moy_elec_heure_pond_ete=conso_moy_elec_heure_pond_ete.reset_index(level='Heure')

    plt.close()
    plt.plot(conso_moy_gaz_heure_pond_ete['Heure'],conso_moy_gaz_heure_pond_ete['Consogaz/hab'],label="Gaz")
    plt.plot(conso_moy_elec_heure_pond_ete['Heure'],conso_moy_elec_heure_pond_ete['Consoelec/hab'],label="Electricité")
    plt.title('Evolution de la consomation par heure en été par habitant')
    plt.xticks(range(0,24))
    plt.xlabel('Heure')
    plt.ylabel('Consommation estivale moyenne par habitant ')
    plt.legend()
    return plt.show()


In [101]:
def plot_conso_gaz_hiver():
    
    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}

    def pond_pop(x):
        return Pop_region[x] 

    consogaz_hiver = conso_gaz_hiver()
    consoelec_hiver = conso_elec_hiver()


    consogaz_hiver['Consogaz/hab']=consogaz_hiver['Consommation brute gaz (W)']/consogaz_hiver['Region'].transform(pond_pop)
    conso_moy_gaz_heure_pond_hiver=consogaz_hiver[['Heure','Consogaz/hab']].groupby('Heure').mean()
    conso_moy_gaz_heure_pond_hiver=conso_moy_gaz_heure_pond_hiver.reset_index(level='Heure')

    consoelec_hiver['Consoelec/hab']=consoelec_hiver['Consommation brute elec (W)']/consoelec_hiver['Region'].transform(pond_pop)
    conso_moy_elec_heure_pond_hiver=consoelec_hiver[['Heure','Consoelec/hab']].groupby('Heure').mean()
    conso_moy_elec_heure_pond_hiver=conso_moy_elec_heure_pond_hiver.reset_index(level='Heure')

    plt.close()
    plt.plot(conso_moy_gaz_heure_pond_hiver['Heure'],conso_moy_gaz_heure_pond_hiver['Consogaz/hab'],label='Gaz')
    plt.plot(conso_moy_elec_heure_pond_hiver['Heure'],conso_moy_elec_heure_pond_hiver['Consoelec/hab'],label='Electricité')
    plt.title('Evolution de la consomation par heure en hiver par habitant')
    plt.xticks(range(0,24))
    plt.xlabel('Heure')
    plt.ylabel('Consommation hivernale moyenne par habitant')
    plt.legend()
    return plt.show()

In [103]:
def plot_conso_elec_ete_hiver():
    consoelec_ete = conso_elec_ete()
    consoelec_hiver = conso_elec_hiver()

    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}

    def pond_pop(x):
        return Pop_region[x]

    consoelec_ete['Consoelec/hab']=consoelec_ete['Consommation brute elec (W)']/consoelec_ete['Region'].transform(pond_pop)
    conso_moy_elec_heure_pond_ete=consoelec_ete[['Heure','Consoelec/hab']].groupby('Heure').mean()
    conso_moy_elec_heure_pond_ete=conso_moy_elec_heure_pond_ete.reset_index(level='Heure')



    consoelec_hiver['Consoelec/hab']=consoelec_hiver['Consommation brute elec (W)']/consoelec_hiver['Region'].transform(pond_pop)
    conso_moy_elec_heure_pond_hiver=consoelec_hiver[['Heure','Consoelec/hab']].groupby('Heure').mean()
    conso_moy_elec_heure_pond_hiver=conso_moy_elec_heure_pond_hiver.reset_index(level='Heure')

    plt.close()
    plt.plot(conso_moy_elec_heure_pond_hiver['Heure'],conso_moy_elec_heure_pond_hiver['Consoelec/hab'],label='Consomation en Hiver',color='r')
    plt.plot(conso_moy_elec_heure_pond_ete['Heure'],conso_moy_elec_heure_pond_ete['Consoelec/hab'], label= 'Consommation en été',color='g')
    plt.legend(loc=7)
    plt.title('Evolution de la consomation en élétricité par heure par habitant (W)')
    plt.xlabel('Heure')
    plt.ylabel('Consommation en éléctricité (par hab)')
    return plt.show()

In [107]:
def plot_conso_gaz_ete_hiver():

    consogaz_ete = conso_gaz_ete()
    consogaz_hiver = conso_gaz_hiver()

    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}

    def pond_pop(x):
        return Pop_region[x]

    consogaz_ete['Consogaz/hab']=consogaz_ete['Consommation brute gaz (W)']/consogaz_ete['Region'].transform(pond_pop)
    conso_moy_gaz_heure_pond_ete=consogaz_ete[['Heure','Consogaz/hab']].groupby('Heure').mean()
    conso_moy_gaz_heure_pond_ete=conso_moy_gaz_heure_pond_ete.reset_index(level='Heure')

    consogaz_hiver['Consogaz/hab']=consogaz_hiver['Consommation brute gaz (W)']/consogaz_hiver['Region'].transform(pond_pop)
    conso_moy_gaz_heure_pond_hiver=consogaz_hiver[['Heure','Consogaz/hab']].groupby('Heure').mean()
    conso_moy_gaz_heure_pond_hiver=conso_moy_gaz_heure_pond_hiver.reset_index(level='Heure')

    plt.close()
    plt.plot(conso_moy_gaz_heure_pond_hiver['Heure'],conso_moy_gaz_heure_pond_hiver['Consogaz/hab'],label= "Hiver",c='r')
    plt.plot(conso_moy_gaz_heure_pond_ete['Heure'],conso_moy_gaz_heure_pond_ete['Consogaz/hab'],label="Été",c='g')
    plt.title('Evolution de la consomation de gaz par heure et par habitant (W)')
    plt.legend()
    plt.xticks(np.arange(0, 24,1))
    plt.ylabel('Consommation gaz hivernale par habitant')
    plt.xlabel('Heure')
    return plt.show()

In [108]:
def conso_we_semaine():

    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}

    def pond_pop(x):
        return Pop_region[x]
    
    conso = nettoyage_bases.conso_quotidienne()

    conso['Population Region']= conso['Region'].transform(pond_pop)
    conso_sem=conso[(conso['Jour semaine']=='Samedi') | (conso['Jour semaine']=='Dimanche') ]
    conso_sem.dropna(subset=['Consommation brute totale (W)'], inplace=True)

    conso_we =conso[(conso['Jour semaine']=='Lundi') | (conso['Jour semaine']=='Dimanche') | (conso['Jour semaine']=='Mardi')| (conso['Jour semaine']=='Mercredi') | (conso['Jour semaine']=='Jeudi')| (conso['Jour semaine']=='Vendredi')]
    conso_we.dropna(subset=['Consommation brute totale (W)'], inplace=True)

    conso_we_moy= conso_we[['Heure','Consommation brute totale (W)']].groupby('Heure').mean()
    conso_we_moy= conso_we_moy.reset_index(level='Heure')

    conso_sem_moy=conso_sem[['Heure','Consommation brute totale (W)']].groupby('Heure').mean()
    conso_sem_moy=conso_sem_moy.reset_index(level='Heure')
    
    plt.close()
    plt.plot(conso_we_moy['Heure'],conso_we_moy['Consommation brute totale (W)'],label= "Consommation en week end",c='g')
    plt.plot(conso_sem_moy['Heure'],conso_sem_moy['Consommation brute totale (W)'],label="Consommation en semaine",c='r')
    plt.legend()
    plt.xticks(np.arange(0, 24,1))
    plt.ylabel('Consommation d énergie (W)')
    plt.xlabel('Heure')
    plt.title('Consommation en énergie en France selon les heures')
    plt.show()

In [ ]:
def conso_gaz_region():

    consogaz = nettoyage_bases.conso_gaz()

    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}


    plt.close()  
    for reg in consogaz['Region'].unique():
        consogaz_reg=consogaz[consogaz['Region']==reg]
        consogaz_reg['Consogaz/hab']=consogaz_reg['Consommation brute gaz (W)']/Pop_region[reg]
        conso_moy_gaz_heure_pond_reg=consogaz_reg[['Heure','Consogaz/hab']].groupby('Heure').mean()
        conso_moy_gaz_heure_pond_reg=conso_moy_gaz_heure_pond_reg.reset_index(level='Heure')


        plt.plot(conso_moy_gaz_heure_pond_reg['Heure'],conso_moy_gaz_heure_pond_reg['Consogaz/hab'],label=reg)
    plt.legend()
    plt.xticks(np.arange(0, 24,1))
    plt.legend(bbox_to_anchor= (1.2,1))
    plt.ylabel('Consommation de Gaz par habitant (W)')
    plt.xlabel('Heure')
    plt.title('Consommation de gaz par habitant en été')
    return plt.show()

In [119]:
def conso_elec_region():

    consoelec = nettoyage_bases.conso_elec()

    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}


    plt.close()  
    for reg in consoelec['Region'].unique():
        consoelec_reg=consoelec[consoelec['Region']==reg]
        consoelec_reg['Consoelec/hab']=consoelec_reg['Consommation brute elec (W)']/Pop_region[reg]
        conso_moy_elec_heure_pond_reg=consoelec_reg[['Heure','Consoelec/hab']].groupby('Heure').mean()
        conso_moy_elec_heure_pond_reg=conso_moy_elec_heure_pond_reg.reset_index(level='Heure')


        plt.plot(conso_moy_elec_heure_pond_reg['Heure'],conso_moy_elec_heure_pond_reg['Consoelec/hab'],label=reg)
    plt.legend()
    plt.xticks(np.arange(0, 24,1))
    plt.legend(bbox_to_anchor= (1.2,1))
    plt.ylabel('Consommation d\'électricité par habitant (W)')
    plt.xlabel('Heure')
    plt.title('Consommation d\'électricité par habitant en été')
    return plt.show()